# Exercise 1: Data cleaning

Before doing actual data analysis, we usually first need to clean the data. 
This might involve steps such as dealing with missing values and encoding categorical variables as integers.
In this exercise, you will perform such steps based on the Titanic passenger data.

1. Load the Titanic data set in `titanic.csv` located in the `data/` folder.
2. Report the number of observations with missing `Age`, for example using [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.isna.html).
3. Compute the average age in the data set. Use the following approaches and compare your results:
    1.  Use pandas's [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) method.
    2.  Convert the `Age` column to a NumPy array using [`to_numpy()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html). Experiment with NumPy's [`np.mean()`](https://numpy.org/doc/2.0/reference/generated/numpy.mean.html) and [`np.nanmean()`](https://numpy.org/doc/2.0/reference/generated/numpy.nanmean.html) to see if you obtain the same results.
4. Replace the all missing ages with the mean age you computed above, rounded to the nearest integer.
   Note that in "real" applications, replacing missing values with sample means is usually not a good idea.
5. Convert this updated `Age` column to integer type using [`astype()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html).
6. Generate a new column `Female` which takes on the value one if `Sex` is equal to `"female"` and zero otherwise. 
   This is called an _indicator_ or _dummy_ variable, and is preferrable to storing such categorical data as strings.
   Delete the original column `Sex`.
7. Save your cleaned data set as `titanic-clean.csv` using [`to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) with `,` as the field separator.
   Tell `to_csv()` to *not* write the `DataFrame` index to the CSV file as it's not needed in this example.

In [28]:
import pandas as pd

# Relative path to data directory
DATA_PATH = '../../data'

# Path to Titanic CSV file
fn = f'{DATA_PATH}/titanic.csv'

# Read in the CSV file, use default separator (comma)
df = pd.read_csv(fn)
missingAgeNumber = df['Age'].isna().sum()

newCol = df['Age']

averageAge = np.nanmean(newCol)

newCol.fillna(averageAge, inplace=True)

newCol = newCol.astype(int)

isFemale =  (df['Sex'] == 'female')

df["Female"] = isFemale.astype(int)

del df["Sex"]

df.to_csv(f'titanic-cleaned.csv')


***
# Exercise 2: Selecting subsets of data

In this exercise, you are asked to select subsets of macroeconomic data for the United States based on some criteria.

1.  Load the annual data from FRED which are located in `FRED_annual.xlsx` 
    in the `data/FRED` folder.

2.  Print the list of columns and the number of non-missing observations.

3.  Since we are dealing with time series data, set the column `Year` as the DataFrame index.

4.  Print all observations for the 1960s decade using at least two different methods.

5.  Using the data in the column `GDP`, compute the annual GDP growth in percent and store it in the column `GDP_growth`. Select the years in which

    1.  GDP growth was above 5%.
    2.  GDP growth was negative, but inflation as still above 5% (such episodes are called "stagflation" since usually negative GDP growth is associated with low inflation).
        
        Use at least two methods to select such years.

    *Hint:* You can compute changes relative to the previous observation using the 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) method.


In [ ]:
DATA_PATH = '../../data'
df = pd.read_excel(f'{DATA_PATH}/FRED/FRED_annual.xlsx')
df.info()

nonMissingObs = df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       70 non-null     int64  
 1   GDP        70 non-null     float64
 2   CPI        70 non-null     float64
 3   UNRATE     70 non-null     float64
 4   FEDFUNDS   70 non-null     float64
 5   INFLATION  69 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 3.4 KB


Year         0
GDP          0
CPI          0
UNRATE       0
FEDFUNDS     0
INFLATION    1
dtype: int64

In [ ]:
df
df.loc[1960:1970]

gdp_growth = df['GDP'].pct_change() * 100



Year
1954         NaN
1955    7.134170
1956    2.134285
1957    2.105564
1958   -0.743367
          ...   
2019    2.583949
2020   -2.163094
2021    6.054984
2022    2.512236
2023    2.887705
Name: GDP, Length: 70, dtype: float64

***
# Exercise 3: Labor market statistics for the US

In this exercise, you are asked to compute some descriptive statistics for the unemployment rate 
and the labor force participation (the fraction of the working-age population in the labor force, i.e., individuals who are either employed or unemployed) for the United States.

1.  Load the monthly time series from FRED which are located in `FRED_monthly.csv` 
    in the `data/FRED` folder.

    *Hint:* You can use `pd.read_csv(..., parse_dates=['DATE'])` to automatically
    parse strings stored in the `DATE` column as dates.

2.  Print the list of columns and the number of non-missing observations.

3.  Since we are dealing with time series data, set the column `DATE` as the DataFrame index. Using the date index, select all observations from the first three months
    of the year 2020.

4.  For the columns `UNRATE` (unemployment rate) and `LFPART` (labor force participation), compute and report 
    the mean, minimum and maximum values for the whole sample. Round your results 
    to one decimal digit.

    *Hint:* You can use the DataFrame methods 
    [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html),
    [`min()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html), and 
    [`max()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html)
    to compute the desired statistics.

    *Hint:* You can use the DataFrame method
    [`round()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html)
    to truncate the number of decimal digits.

5.  You are interested in how the average unemployment rate evolved over the last 
    few decades. 

    -   Add a new column `Decade` to the DataFrame which contains the 
        starting year for each decade (e.g., this value should be 1950
        for the years 1950-1959, and so on).

        *Hint:* The decade can be computed from the column `Year` using 
        truncated integer division:
        ```python
        df['Year'] // 10 * 10
        ```

    -   Write a loop to compute and report the average unemployment rate (column `UNRATE`) 
        for each decade.

        Include only the decades from 1950 to 2010 for which you have all
        observations. 

In [67]:
DATA_PATH = '../../data'

file = (f'{DATA_PATH}/FRED/FRED_monthly.csv')
df = pd.read_csv(file)

nonMissingObs = df.isna().sum()

df.set_index('DATE', inplace=True)
data2023 = df.loc['2023-01-01':'2023-03-01']

unemploymentRateMin = data2023['UNRATE'].min()
unemploymentRateMax = data2023['UNRATE'].max()
unemploymentRateMean = data2023['UNRATE'].mean()

print(f"The minimum unemployment rate in the first three months of 2023 was {unemploymentRateMin:.1f}%")
print(f"The maximum unemployment rate in the first three months of 2023 was {unemploymentRateMax:.1f}%")
print(f"The average unemployment rate in the first three months of 2023 was {unemploymentRateMean:.2f}%")

labourForcePartMin =data2023['LFPART'].min()
labourForcePartMax =data2023['LFPART'].max()
labourForcePartMean =data2023['LFPART'].mean()

print(f"The minimum labour force participation rate in the first three months of 2023 was {labourForcePartMin:.1f}%")
print(f"The maximum labour force participation rate in the first three months of 2023 was {labourForcePartMax:.1f}%")
print(f"The average labour force participation rate in the first three months of 2023 was {labourForcePartMean:.2f}%")


The minimum unemployment rate in the first three months of 2023 was 3.5%
The maximum unemployment rate in the first three months of 2023 was 3.6%
The average unemployment rate in the first three months of 2023 was 3.53%
The minimum labour force participation rate in the first three months of 2023 was 62.4%
The maximum labour force participation rate in the first three months of 2023 was 62.6%
The average labour force participation rate in the first three months of 2023 was 62.50%


In [72]:
df['Decade'] = df['Year'] // 10*10
decades = [1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
for year in df['Decade']:
    for decade in decades:
        unrateMean = df['UNRATE'][decade:decade+10].mean()
        print(f"The average unemployment rate in the {decade}s was {unrateMean:.2f}%")



The average unemployment rate in the 1950s was nan%
The average unemployment rate in the 1960s was nan%
The average unemployment rate in the 1970s was nan%
The average unemployment rate in the 1980s was nan%
The average unemployment rate in the 1990s was nan%
The average unemployment rate in the 2000s was nan%
The average unemployment rate in the 2010s was nan%
The average unemployment rate in the 2020s was nan%
The average unemployment rate in the 1950s was nan%
The average unemployment rate in the 1960s was nan%
The average unemployment rate in the 1970s was nan%
The average unemployment rate in the 1980s was nan%
The average unemployment rate in the 1990s was nan%
The average unemployment rate in the 2000s was nan%
The average unemployment rate in the 2010s was nan%
The average unemployment rate in the 2020s was nan%
The average unemployment rate in the 1950s was nan%
The average unemployment rate in the 1960s was nan%
The average unemployment rate in the 1970s was nan%
The average 

***
# Exercise 4: Working with string data (advanced)

Most of the data we deal with contain strings, i.e., text data (names, addresses, etc.). Often, such data is not in the format needed for analysis, and we have to perform additional string manipulation to extract the exact data we need. This can be achieved using the pandas [string methods](https://pandas.pydata.org/docs/user_guide/text.html#string-methods).

To illustrate, we use the Titanic data set for this exercise.

1.  Load the Titanic data and restrict the sample to men. (This simplifies the task. Women in this data set have much more complicated names as they contain both their husband's and their maiden name)
2.  Print the first five observations of the `Name` column. As you can see, the data is stored in the format _"Last name, Title First name"_ where title is something like Mr., Rev., etc.
3. Split the `Name` column by `,` to extract the last name and the remainder as separate columns. You can achieve this using the [`partition()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.partition.html#pandas.Series.str.partition) string method.
4. Split the remainder (containing the title and first name) using the space character `" "` as separator to obtain individual columns for the title and the first name.
5. Store the three data series in the original `DataFrame` (using the column names `FirstName`, `LastName` and `Title`) and delete the `Name` column which is no longer needed.
6. Finally, extract the ship deck from the values in `Cabin`. The ship deck is the first character in the string stored in `Cabin` (A, B, C, ...). You extract the first character using the 
[`get()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get.html#pandas.Series.str.get) string method. Store the result in the column `Deck`.

*Hint*: Pandas's string methods can be accessed using the `.str` attribute. For example, to partition values in the column `Name`, you need to use
```python
df['Name'].str.partition()
```
